In [2]:
import numpy as np
import numpy.linalg as npl
# read in the data
dataTrain = np.loadtxt('IDSWeedCropTrain.csv', delimiter = ',')
dataTest = np.loadtxt('IDSWeedCropTest.csv', delimiter = ',')
# split input variables and labels
XTrain = dataTrain[:,:-1]
YTrain = dataTrain[:,-1]
XTest = dataTest[:,:-1]
YTest = dataTest[:,-1]

In [35]:
#Exercise 1
def predict_one(Xvector, k, XTrain, YTrain):
    '''Takes in a single vector of 13 values, and makes a prediction using X and Y training data for k nearest neighbors'''
    distances = npl.norm(XTrain - Xvector, axis = 1) #create an array of dsitances between our vector and the training data
    neighbor_index = select_neighbors(distances, k) #find the index of the k enarest neighbors
    neighbors = YTrain[neighbor_index] #calculate the Y values of the k nearest neighbors
    if sum(neighbors) > k/2: #if the majority of k values are 1, predict 1 else predict 0
        return 1
    else:
        return 0

def select_neighbor_index(distances, k):
    '''Take in a vector of distances and calculate the k nearest neighbor indexes for them'''
    sorted_distances = np.sort(distances) #create a sorted array
    neighbor_distances = sorted_distances[:k] #take the first k values
    neighbor_index = []
    for i in neighbor_distances: #calculate the indexes for the first k values
        ind = np.where(distances == i)[0][0]
        neighbor_index.append(ind)
    return neighbor_index
    
def knn_predict(XTest, k = 1, XTrain = XTrain, YTrain = YTrain):
    '''Takes in an array of 13 values per row, and returns an array of predictions'''
    Ypred = []
    for i in XTest:
        Ypred.append(predict_one(i, k, XTrain, YTrain))
    Ypred = np.array(Ypred)
    return Ypred

In [36]:
from sklearn.metrics import accuracy_score
# given classifier called knn , compute the accuracy on the testset
accTest = accuracy_score(YTest, knn_predict(XTest))
print("Accuracy: " + str(accTest))

Accuracy: 0.945993031358885


In [50]:
#Exercise 2
from sklearn.model_selection import KFold

def cross_validate(k, XTrain, YTrain, XTest, YTest):
    '''For a value k, perform 5 fold cross validation for k nearest neighbors'''
    accuracy_list = []
    # create indices for CV
    cv = KFold (n_splits = 5)
    # loop over CV folds
    for train, test in cv.split(XTrain):
        XTrainCV, XTestCV, YTrainCV, YTestCV = XTrain[train], XTrain[test], YTrain[train], YTrain[test]
        accTest = accuracy_score(YTestCV, knn_predict(XTestCV, k, XTrainCV, YTrainCV))
        accuracy_list.append(accTest)
    return np.mean(accuracy_list)

def k_best(k_list, XTrain=XTrain, YTrain=YTrain, XTest=XTest, YTest=YTest):
    '''Given a list of ks, perform 5 fold cross validation for each k and return the best k'''
    k_loss = []
    for k in k_list:
        accuracy = cross_validate(k, XTrain, YTrain, XTest, YTest)
        loss = 1 - accuracy
        k_loss.append(loss)
        print("Loss for "+ str(k) + " neighbors: " + str(loss))
    ind = k_loss.index(min(k_loss))
    best_k = k_list[ind]
    print("Best k: " + str(best_k))
    return best_k

k_list = [1,3,5,7,9,11]
k_best(k_list)

Loss for 1 neighbors: 0.04600000000000004
Loss for 3 neighbors: 0.037000000000000144
Loss for 5 neighbors: 0.04399999999999993
Loss for 7 neighbors: 0.050000000000000044
Loss for 9 neighbors: 0.05500000000000005
Loss for 11 neighbors: 0.05600000000000005
Best k: 3


3

In [49]:
#Exercise 3
best_acc = accuracy_score(YTest, knn_predict(XTest, k = 3))
print("Accuracy: " + str(best_acc))

Accuracy: 0.9494773519163763


In [ ]:
#Exercise 4